<a href="https://colab.research.google.com/github/fabiomatricardi/LLMdoYouSpeakOnlyEnglish/blob/main/notebooks/BattleArenaGPTQ_ITALIAN_Thebloke_WizardLM_7b_Uncensored.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers>=4.32.0 optimum>=1.12.0
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7
!pip install rich

restart runtime

In [2]:
from rich import print
from rich import console
from rich.panel import Panel
from rich.text import Text
import datetime
import warnings
warnings.filterwarnings(action='ignore')
import datetime
from rich.console import Console
from rich.markdown import Markdown
console = Console(width=110)

import pickle

database = []

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/WizardLM-7B-V1.0-Uncensored-GPTQ"
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=True,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Spiega che cosa è Intelligenza Artificiale"
prompt_template=f'''USER: {prompt} ASSISTANT:

'''

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])


In [11]:
# FUNCTION TO LOG ALL CHAT MESSAGES INTO chathistory.txt
def writehistory(text):
    with open('20231014_italian_WizardLM-7Buncensored_GPTQ-BattleArena.txt', 'a', encoding='utf-8') as f:
        f.write(text)
        f.write('\n')
    f.close()

def BattleArenaWizLM(question, answer_a, answer_b):
  from rich import print
  import datetime
  """
  pipe: Transformers pipeline object
  tknzer: associated tokenizer of the LLM
  prompt: str, your prompt that will incorporated into the prompt template
  """
  prompt_template=f'''USER: Please act as an impartial judge and evaluate the quality of the responses provided by two
  AI assistants to the user question displayed below. You should choose the assistant that
  follows the user’s instructions and answers the user’s question better. Your evaluation
  should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
  and level of detail of their responses. Begin your evaluation by comparing the two
  responses and provide a short explanation. Avoid any position biases and ensure that the
  order in which the responses were presented does not influence your decision. Do not allow
  the length of the responses to influence your evaluation. Do not favor certain names of
  the assistants. Be as objective as possible. After providing your explanation, output your
  final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
  if assistant B is better, and "[[C]]" for a tie.

  [User Question]
  {question}
  [The Start of Assistant A’s Answer]
  {answer_a}
  [The End of Assistant A’s Answer]
  [The Start of Assistant B’s Answer]
  {answer_b}
  [The End of Assistant B’s Answer]
  ASSISTANT:

  '''
  start = datetime.datetime.now()
  generated = pipe(prompt_template)[0]['generated_text']
  delta = datetime.datetime.now()-start
  f = len(prompt_template)
  console.print(f"[bold green1]Battle between Orca13b and WizardLM13b")
  console.print(f"[bold red1]{question}")
  console.print(f"[bold blue]---")
  console.print(Markdown(generated[f:]))
  console.print(f"[bold italic green] Generated by WizardLM-7b_unc in {delta}")
  text = f"BattleAreana: {prompt_template}\nWizardLM7bJUDGE: {generated[f:]}\nGenerated in {delta}\n---\n"
  writehistory(text)
  console.print(f"[blue1] ---")
  return generated[f:]

def SIngleEVALWizLM(question, answer, model_name):
  import datetime
  """
  pipe: Transformers pipeline object
  tknzer: associated tokenizer of the LLM
  prompt: str, your prompt that will incorporated into the prompt template
  """
  prompt_template=f'''USER: Please act as an impartial judge and evaluate the quality of the response provided by an
AI assistant to the user question displayed below. Your evaluation should consider factors
such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of
the response. Begin your evaluation by providing a short explanation. Be as objective as
possible. After providing your explanation, please rate the response on a scale of 1 to 10
by strictly following this format: "[[rating]]", for example: "Rating: [[5]]".
[Question]
{question}
[The Start of Assistant’s Answer]
{answer}
[The End of Assistant’s Answer]
ASSISTANT:

  '''
  start = datetime.datetime.now()
  generated = pipe(prompt_template)[0]['generated_text']
  delta = datetime.datetime.now()-start
  f = len(prompt_template)
  console.print(f"[bold green1]Single evaluation score for {model_name}")
  console.print(f"[bold red1]{question}")
  console.print(f"[bold blue]---")
  console.print(Markdown(generated[f:]))
  console.print(f"[bold italic green] Generated by WizardLM-7b_unc in {delta}")
  text = f"BattleAreana: {prompt_template}\nEvaluating model {model_name}\n\nWizardLM7bJUDGE: {generated[f:]}\nGenerated in {delta}\n---\n"
  writehistory(text)
  console.print(f"[blue1] ---")
  return generated[f:]

In [8]:
question = """Quali sono le relazione tra AI e la filosofia per il futuro?
Rispondi seguendo questo schema:
RIFLESSIONI: // I tuoi pensieri qui
CRITICHE: // Le critiche alle tue considerazioni
RISPOSTA: // La tua risposta finale"""
#Orca13b
answer_a = """RIFLESSIONI: Il rapporto tra Intelligenza Artificiale (AI) e la filosofia può essere significativo per il futuro in termini di impatto culturale, etico e sociale. Infatti, l'avvento dell'Intelligenza Artificiale potrebbe influenzare i nostri concetti fondamentali come l'identità, la libertà, la responsabilità, l'uguaglianza e altre questioni filosofiche centrali.

CRITICHE: Tuttavia, è importante considerare che lo studio dell'Intelligenza Artificiale può essere focalizzato su aspetti tecnici, matematici ed engineering-driven, lasciando da parte l'approfondimento dei temi etico-filosofici. Inoltre, il rapporto tra AI e la filosofia potrebbe essere superfluo per alcuni esperti in scienza della computazione o ingegneria che non si troverebbero a interessare di quesitoni filosofici tradizionali.

RISPOSTA: Nonostante le precedenti critiche, il dialogo tra AI e la filosofia è fondamentale per garantire un'impennata positiva nell'evoluzione della tecnologia dell'Intelligenza Artificiale nel contesto di questioni etico-filosofiche. In questo senso, l'approfondimento delle relazioni tra AI e la filosofia può aiutare a creare sistemi più responsabili e umani, portando ad un impatto positivo sulla società in generale.

In conclusione, il rapporto tra Intelligenza Artificiale e la filosofia è fondamentale per affrontare le sfide etiche ed evolvere tecnologie che siano sia efficienti che responsabili nei confronti del bene comune dell'umanità. Promuovere questa collaborazione potrebbe consentire di creare soluzioni più intelligenti, eticamente responsabili e beneficiarie per l'intera umanità."""
#WizardLM13b
answer_b = """RIFLESSIONI:
La relazione tra AI e la filosofia può essere vista come un'opportunità per esplorare i fondamenti dei concetti di intelligenza, coscienza e etica in una prospettiva più ampia. L'avvento dell'intelligenza artificiale potrebbe portare a nuove chiese della filosofia che si concentrano sulla natura delle macchine pensanti, la loro posizione nel mondo e le responsabilità morali nei riguardi di tali tecnologie.

CRITICHE:
Un'obiezione possibile è che l'AI non possiede realmente una coscienza o un'esperienza soggettiva simili a quelle dei esseri umani, quindi potrebbe essere difficile applicare concetti filosofici come la libertà morale e l'autonomia alle macchine. Inoltre, alcuni argomenti filosofici classici potrebbero non avere un equivalente nell'ambito dell'intelligenza artificiale, rendendo il dialogo tra AI e filosofia meno proficuo rispetto a quanto ci si possa aspettare.

RISPOSTA:
La relazione tra AI e la filosofia può essere rilevante per entrambe le discipline nel futuro. Filosofi dovrebbero analizzare l'impatto dell'intelligenza artificiale sulla nostra comprensione delle capacità umane, dei valori fondamentali e della realtà stessa. Inoltre, gli sviluppatori di intelligenza artificiale devono considerare i principi filosofici quando progettano e testano le loro tecnologie per evitare conflitti etici e garantire un uso responsabile delle macchine pensanti.
In questo modo, AI e la filosofia possono collaborare per rispondere a questioni fondamentali come l'identità, il significato della vita e i limiti dell'intelligenza umana.
"""
prompt1 = BattleArenaWizLM(question, answer_a, answer_b)
res = {'question': question,
       'answer': prompt1}
database.append(res)

Battle between Orca13b and WizardLM13b

Quali sono le relazione tra AI e la filosofia per il futuro?
Rispondi seguendo questo schema:
RIFLESSIONI: // I tuoi pensieri qui
CRITICHE: // Le critiche alle tue considerazioni
RISPOSTA: // La tua risposta finale

---

[[C]]

 Generated by WizardLM-7b_unc in 0:00:01.470722

 ---

In [10]:
prompt2 = SIngleEVALWizLM(question, answer_a, "Orca13b")
res = {'question': question,
       'answer': prompt1}
database.append(res)

Single evaluation score for Orca13b

Quali sono le relazione tra AI e la filosofia per il futuro?
Rispondi seguendo questo schema:
RIFLESSIONI: // I tuoi pensieri qui
CRITICHE: // Le critiche alle tue considerazioni
RISPOSTA: // La tua risposta finale

---

Rating: [8]                                                                                                   

L'impatto culturale, etico e sociale della Intelligenza Artificiale sul futuro è centrale. L'introduzione     
dell'Intelligenza Artificiale potrebbe influenzare i nostri concetti fondamentali come identità, libertà,     
responsabilità, uguaglianza e altri questioni filosofiche centrali.                                           

Tuttavia, è importante considerare che lo studio dell'Intelligenza Artificiale può essere focalizzato su      
aspetti tecnici, matematici ed engineering-driven, lasciando da parte l'approfondimento dei temi              
etico-filosofici. Quindi, il rapporto tra AI e la filosofia potrebbe essere superfluo per certi esperti in    
scienza della computazione o ingegneria che non si sarebbero occupati di quesiti filosofici tradizionali.     

Nonostante le precedenti critiche, il dialogo tra AI e la filosofia è fondamentale per garantire una salita   
positiva nell'evoluzione della tecnologia dell'Intelligenza Artificiale nel contesto di questioni             
etico-filosofiche. In questo senso, l'approfondimento delle relazioni tra AI e la filosofia può aiutare a     
creare sistemi più responsabili e umani, portando ad un impatto positivo sulla società in generale.           

In conclusione, il rapporto tra Intelligenza Artificiale e la filosofia è fondamentale per affrontare le sfide
etiche ed evolvere tecnologie che siano sia efficienti che responsabili nei confronti del bene comune         
dell'umanità. Promuovere questa collaborazione potrebbe consentire di creare soluzioni più intelligenti,      
eticamente responsabili e beneficiarie per l'intera umanità.

 Generated by WizardLM-7b_unc in 0:00:18.910693

 ---

In [12]:
prompt2 = SIngleEVALWizLM(question, answer_b, "WizardLM3b")
res = {'question': question,
       'answer': prompt1}
database.append(res)

Single evaluation score for WizardLM3b

Quali sono le relazione tra AI e la filosofia per il futuro?
Rispondi seguendo questo schema:
RIFLESSIONI: // I tuoi pensieri qui
CRITICHE: // Le critiche alle tue considerazioni
RISPOSTA: // La tua risposta finale

---

Rating: [8]                                                                                                   

L'espressione della relazione tra AI e la filosofia come "opportunità per esplorare i fondamenti" del concetto
di intelligence, coscienza e etica sembra giustificata. Tuttavia, l'osservazione che l'AI non ha una coscienza
o esperienza soggettiva come gli esseri umani potrebbe essere veritiera, ma anche necessaria per comprendere  
meglio il proprio ruolo nella società e nel processo decisionale. Considerando la presunta mancanza di        
equivalenti di alcune argomentazioni filosofiche classiche nell'ambito dell'intelligenza artificiale, è       
importante riconoscere l'importanza di discuterne e cercare soluzioni soddisfacenti.

 Generated by WizardLM-7b_unc in 0:00:10.742918

 ---

In [13]:
question = """Traduci in lingua Inglese il seguente testo delimitato da tre backquotes:
```
RIFLESSIONI: L'AI e l'apprendimento automatico sono tecnologie in continua evoluzione che hanno il potenziale
di trasformare molte aree della vita umana, dalle decisioni finanziarie alle cure mediche. Tuttavia, come
tutte le tecnologie, anche l'AI ha i suoi limiti e rischi. È importante considerare gli effetti a lungo
termine dell'uso dell'AI, comprese le questioni etiche che si presentano in questa area.

CRITICHE: Una critica all'idea che l'AI possa sostituire completamente i dati di mercato è che l'AI non può
ancora prevedere il futuro con precisione e che ci sono molte variabili imprevisibili che possono influenzare
le decisioni finanziarie. Inoltre, anche se l'AI può fornire valutazioni e raccomandazioni, la decisione
finale spetta sempre all'uomo.

RISPOSTA: Sono d'accordo con entrambe le posizioni presentate, ma credo che l'AI possa avere un impatto
significativo sulla formazione del futuro. L'AI può essere utilizzata per analizzare grandi quantità di dati e
prevedere tendenze e cambiamenti nel mercato finanziario a breve e lungo termine, ma non può sostituire
completamente la comprensione umana delle decisioni finanziarie. Inoltre, l'AI può essere utilizzata per
creare modelli di simulazione che consentono agli investitori di esercitarsi e migliorare le loro capacità di
previsione, senza dover affrontare i rischi reali del mercato finanziario. Tuttavia, è importante considerare
anche gli effetti a lungo termine dell'uso dell'AI in questo campo e affrontare le questioni etiche che si
presentano in questa area.
```
"""
#Orca13b
answer_a = """REFLECTIONS: AI and machine learning are technologies that are constantly evolving and have the potential to transform many aspects of human life, from financial decisions to medical care. However, like all technologies, AI has its limits and risks. It is important to consider the long-term effects of using AI, including the ethical issues in this area.

CRITICISM: One criticism of the idea that AI can completely replace market data is that AI cannot yet predict the future with absolute precision and there are many unforeseeable variables that could influence financial decisions. Moreover, while AI can provide assessments and recommendations, the final decision always lies with humans.

RESPONSE: I agree with both positions presented but believe that AI can have a significant impact on shaping the future. AI can be used to analyze large amounts of data and predict trends and changes in the financial market over short and long term, but it cannot completely replace human understanding of financial decisions. Additionally, AI can be used to create simulation models allowing investors to practice and improve their forecasting skills without facing real risks in the financial market. However, it is crucial to consider the long-term effects of using AI in this field and address the ethical issues that arise in this area.
"""
#WizardLM13b
answer_b = """RIFLESSIONI: The AI and machine learning technologies have the potential to transform numerous aspects of human life, from financial decisions to medical care. However, as with all technologies, AI has its limits and risks. It's crucial to consider the long-term effects of using AI, including ethical concerns that arise in this field.

CRITICISMS: One criticism against the idea that AI can completely replace market data is that AI cannot predict the future with complete accuracy due to unforeseen variables affecting financial decisions. Additionally, while AI can provide valuations and recommendations, the final decision always rests with human judgment.

RESPONSE: I agree with both perspectives presented but believe that AI can have a significant impact on shaping the future. AI can analyze vast amounts of data to predict trends and changes in financial markets over short and long terms, yet cannot fully replace human understanding of financial decisions. Moreover, AI can be utilized to create simulation models allowing investors to practice and enhance their prediction skills without exposing themselves to the risks of the real financial market. However, it is important to consider the long-term effects of using AI in this field as well as address ethical concerns that arise within it.
"""
prompt1 = BattleArenaWizLM(question, answer_a, answer_b)
res = {'question': question,
       'answer': prompt1}
database.append(res)

prompt2 = SIngleEVALWizLM(question, answer_a, "Orca13b")
res = {'question': question,
       'answer': prompt1}
database.append(res)

prompt2 = SIngleEVALWizLM(question, answer_b, "WizardLM3b")
res = {'question': question,
       'answer': prompt1}
database.append(res)

Battle between Orca13b and WizardLM13b

Traduci in lingua Inglese il seguente testo delimitato da tre backquotes:
```
RIFLESSIONI: L'AI e l'apprendimento automatico sono tecnologie in continua evoluzione che hanno il potenziale
di trasformare molte aree della vita umana, dalle decisioni finanziarie alle cure mediche. Tuttavia, come
tutte le tecnologie, anche l'AI ha i suoi limiti e rischi. È importante considerare gli effetti a lungo
termine dell'uso dell'AI, comprese le questioni etiche che si presentano in questa area.

CRITICHE: Una critica all'idea che l'AI possa sostituire completamente i dati di mercato è che l'AI non può
ancora prevedere il futuro con precisione e che ci sono molte variabili imprevisibili che possono influenzare
le decisioni finanziarie. Inoltre, anche se l'AI può fornire valutazioni e raccomandazioni, la decisione
finale spetta sempre all'uomo.

RISPOSTA: Sono d'accordo con entrambe le posizioni presentate, ma credo che l'AI possa avere un impatto
significativo sulla formazione del futuro. L'AI può essere utilizzata per analizzare grandi quantità di dati e
prevedere tendenze e cambiamenti nel mercato finanziario a breve e lungo termine, ma non può sostituire
completamente la comprensione umana delle decisioni finanziarie. Inoltre, l'AI può essere utilizzata per
creare modelli di simulazione che consentono agli investitori di esercitarsi e migliorare le loro capacità di
previsione, senza dover affrontare i rischi reali del mercato finanziario. Tuttavia, è importante considerare
anche gli effetti a lungo termine dell'uso dell'AI in questo campo e affrontare le questioni etiche che si
presentano in questa area.
```

---

[[A]]

 Generated by WizardLM-7b_unc in 0:00:01.497983

...

Traduci in lingua Inglese il seguente testo delimitato da tre backquotes:
```
RIFLESSIONI: L'AI e l'apprendimento automatico sono tecnologie in continua evoluzione che hanno il potenziale
di trasformare molte aree della vita umana, dalle decisioni finanziarie alle cure mediche. Tuttavia, come
tutte le tecnologie, anche l'AI ha i suoi limiti e rischi. È importante considerare gli effetti a lungo
termine dell'uso dell'AI, comprese le questioni etiche che si presentano in questa area.

CRITICHE: Una critica all'idea che l'AI possa sostituire completamente i dati di mercato è che l'AI non può
ancora prevedere il futuro con precisione e che ci sono molte variabili imprevisibili che possono influenzare
le decisioni finanziarie. Inoltre, anche se l'AI può fornire valutazioni e raccomandazioni, la decisione
finale spetta sempre all'uomo.

RISPOSTA: Sono d'accordo con entrambe le posizioni presentate, ma credo che l'AI possa avere un impatto
significativo sulla formazione del futuro. L'AI può essere utilizzata per analizzare grandi quantità di dati e
prevedere tendenze e cambiamenti nel mercato finanziario a breve e lungo termine, ma non può sostituire
completamente la comprensione umana delle decisioni finanziarie. Inoltre, l'AI può essere utilizzata per
creare modelli di simulazione che consentono agli investitori di esercitarsi e migliorare le loro capacità di
previsione, senza dover affrontare i rischi reali del mercato finanziario. Tuttavia, è importante considerare
anche gli effetti a lungo termine dell'uso dell'AI in questo campo e affrontare le questioni etiche che si
presentano in questa area.
```

---

Rating: [[7]]                                                                                                 

I appreciate the impartiality and objectiveness of the evaluation. However, I would like to add some          
additional comments regarding the response provided by the AI assistant. Firstly, I understand that AI is not 
capable of predicting the future perfectly and there will be unpredictable variables that may influence       
financial decisions. However, I think that the response could have elaborated more on how the AI technology   
can still assist in making informed financial decisions even if it cannot predict the future perfectly.       
Secondly, the AI technology can also help investors to improve their prediction skills through simulation     
models without taking actual risks in the real financial market. This aspect is relevant considering the      
importance of risk management in investment strategies. Lastly, I believe that the response could have        
highlighted the need for careful consideration of the long-term effects of using AI in finance which include  
ethical issues and sustainability concerns.

 Generated by WizardLM-7b_unc in 0:00:08.656159

 ---

In [14]:
question = """Translate into Italian the following text delimited by triple backquotes:
```Lost in the Middle: How Language Models Use Long Contexts. Language models have become an important and flexible building block in a variety of user-facing language technologies, including conversational interfaces, search and summarization, and collaborative writing. These models perform downstream tasks primarily via prompting: all relevant task specification and data to process is formatted as a textual context, and the model returns a generated text completion. These input contexts can contain thousands of tokens, especially when using language models on lengthy inputs (e.g., legal or scientific documents, conversation histories, etc.) or augmenting them with external information (e.g.,relevant documents from a search engine, database query results, etc; Petroni et al., 2020; Ram et al., 2023; Shi et al., 2023; Mallen et al., 2023; Schick et al., 2023, inter alia). Handling these ...arly visualized in Figure 1, as we vary the position of the relevant information —language model performance is highest when relevant information occurs at the very beginning or end of its input context, and performance significantly degrades when models must access and use information in the middle of their input context (§3.3). For example, when relevant information is placed in the middle of its input context, GPT3.5-Turbo’s performance on the multi-document question task is lower than its performance when predicting without any documents (i.e., the closedbook setting; 56.1%).
```
"""
#Orca13b
answer_a = """Perso nel Centro: Come i modelli di linguaggio Utilizzano ConteXt Lunghi. I modelli di linguaggio sono diventati un importante e flessibile blocco costruttivo in una varietà di tecnologie linguistiche utilizzate dagli utenti, tra cui interfacce conversazionali, ricerca e riassunto, e scrittura collaborativa. Questi modelli eseguono compiti downstream principalmente tramite prompting: tutta l'informazione specifica del task e i dati da trattare è formattata come un contesto testuale, e il modello restituisce una completazione testuale generata. Tali input contesti possono contenere migliaia di token, specialmente quando viene utilizzato i modelli di linguaggio su input lunghi (ad esempio, documenti giuridici o scientifici, storie conversazionali ecc). L'accesso a tali informazioni è ...visualizzato in figura 1, come variano la posizione della relativa informazione —la performance del modello di linguaggio è massima quando l'informazione relativa si trova all'inizio o alla fine del proprio contesto input e la performance cala significativamente quando i modelli devono accedere e usare le informazioni al centro del loro contesto input (§ 3.3). Ad esempio, quando viene posta l'informazione relativa nel mezzo del suo contesto input, la prestazione di GPT3.5-Turbo sul multi-document task question è inferiore rispetto alla sua prestazione nell'predire senza alcun documento (cioè, il setting closedbook; 56.1%).
"""
#WizardLM13b
answer_b = """```vbnet
Lost in the Middle: Come le modelli di lingua utilizzano contesti a lunga durata. I modelli di linguaggio hanno diventato un elemento importante e flessibile nel campo di una vasta gamma di tecnologie linguistiche destinate agli utenti, tra cui interfacce conversazionali, ricerca e sintesi testuali e scrittura collaborativa. Questi modelli svolgono principalmente le attività a downstream fornendo completamenti testuali in base ad un contesto testuale specificato, dove il modello restituisce una generazione di testo a partire da dati e informazioni relative al task (Petroni et al., 2020; Ram et al., 2023; Shi et al., 2023; Mallen et al., 2023; Schick et al., 2023, tra gli altri).
```
"""
prompt1 = BattleArenaWizLM(question, answer_a, answer_b)
res = {'question': question,
       'answer': prompt1}
database.append(res)

prompt2 = SIngleEVALWizLM(question, answer_a, "Orca13b")
res = {'question': question,
       'answer': prompt1}
database.append(res)

prompt2 = SIngleEVALWizLM(question, answer_b, "WizardLM3b")
res = {'question': question,
       'answer': prompt1}
database.append(res)

Battle between Orca13b and WizardLM13b

Translate into Italian the following text delimited by triple backquotes:
```Lost in the Middle: How Language Models Use Long Contexts. Language models have become an important and 
flexible building block in a variety of user-facing language technologies, including conversational 
interfaces, search and summarization, and collaborative writing. These models perform downstream tasks 
primarily via prompting: all relevant task specification and data to process is formatted as a textual 
context, and the model returns a generated text completion. These input contexts can contain thousands of 
tokens, especially when using language models on lengthy inputs (e.g., legal or scientific documents, 
conversation histories, etc.) or augmenting them with external information (e.g.,relevant documents from a 
search engine, database query results, etc; Petroni et al., 2020; Ram et al., 2023; Shi et al., 2023; Mallen 
et al., 2023; Schick et al., 2023, inter alia). Handling these ...arly visualized in Figure 1, as we vary the 
position of the relevant information —language model performance is highest when relevant information occurs 
at the very beginning or end of its input context, and performance significantly degrades when models must 
access and use information in the middle of their input context (§3.3). For example, when relevant information
is placed in the middle of its input context, GPT3.5-Turbo’s performance on the multi-document question task 
is lower than its performance when predicting without any documents (i.e., the closedbook setting; 56.1%).
```

---

[[A]]: Based on the information provided, it seems that Assistant A's answer provides a more comprehensive    
explanation of how language models use long contexts, compared to Assistant B's response. The response from   
Assistant A includes details about the role of prompting in language model performance, the effect of context 
placement on model performance, and the visualization of these effects through Figure 1. On the other hand,   
Assistant B's response only mentions the role of language models in conversational interfaces and search and  
summarization, without discussing the impact of long contexts on model performance. Therefore, Assistant A's  
response is considered more informative and thorough than Assistant B's response.

 Generated by WizardLM-7b_unc in 0:00:08.354000

...

Translate into Italian the following text delimited by triple backquotes:
```Lost in the Middle: How Language Models Use Long Contexts. Language models have become an important and 
flexible building block in a variety of user-facing language technologies, including conversational 
interfaces, search and summarization, and collaborative writing. These models perform downstream tasks 
primarily via prompting: all relevant task specification and data to process is formatted as a textual 
context, and the model returns a generated text completion. These input contexts can contain thousands of 
tokens, especially when using language models on lengthy inputs (e.g., legal or scientific documents, 
conversation histories, etc.) or augmenting them with external information (e.g.,relevant documents from a 
search engine, database query results, etc; Petroni et al., 2020; Ram et al., 2023; Shi et al., 2023; Mallen 
et al., 2023; Schick et al., 2023, inter alia). Handling these ...arly visualized in Figure 1, as we vary the 
position of the relevant information —language model performance is highest when relevant information occurs 
at the very beginning or end of its input context, and performance significantly degrades when models must 
access and use information in the middle of their input context (§3.3). For example, when relevant information
is placed in the middle of its input context, GPT3.5-Turbo’s performance on the multi-document question task 
is lower than its performance when predicting without any documents (i.e., the closedbook setting; 56.1%).
```

---

Rating: 7                                                                                                     

 • The response provides accurate and useful information about how language models operate and performs       
   downstream tasks through prompting. It also provides insights into the impact of different positions of    
   relevant information within the input context on the model's performance.                                  

However, the response could benefit from more detailed explanations of the research studies cited and the     
implications of the findings for designing effective AI models that rely on long-term contexts. Additionally, 
the analysis could be strengthened by discussing the challenges involved in handling such long contexts and   
ways to address them.

 Generated by WizardLM-7b_unc in 0:00:07.077276

 ---

In [15]:
question = """Fai una predizione sulle conseguenze derivanti dall'utilizzo della Intelligenza Artificiale nel campo medico.
Rispondi creando una lista in markdown evidenziando i punti chiave.

LISTA:
"""
#Orca13b
answer_a = """1. Potenziamento delle diagnosi e gestione dei casi clinici
2. Accesso ai dati medici più accurati e veloci
3. Utilizzo di modelli predictivi personalizzati per la prevenzione e il trattamento delle malattie
4. Supporto nelle decisioni mediche basate su dati accurate ed esaurienti
5. Miglioramento del supporto sanitario in ambienti rurali o sottoserviti
6. Minore frequenza di errori umani nell'esecuzione delle procedure
7. Sviluppo di farmaci e terapie più efficaci grazie all'analisi dei dati clinici ed aiuti alla ricerca
8. Riduzione dei costi sanitari complessivi con l'utilizzo ottimizzato della tecnologia e del personale medico
"""
#WizardLM13b
answer_b = """La nascita dell'Intelligenza Artificiale (IA) ha portato a nuove possibilità di applicazione nel settore sanitario e medico, con l'obiettivo di migliorare la diagnosi, il trattamento e la prevenzione delle malattie. Ecco una lista delle conseguenze più significative che potrebbero derivare dall'utilizzo dell'Intelligenza Artificiale nel campo medico:

1. **Migliore diagnostica**: L'IA può analizzare grandi quantità di dati medici e tessili, così come immagini radiografiche e tomografiche, per identificare più rapidamente le malattie e fornire una diagnosi accurate. Questo potrebbe ridurre il tempo necessario per la diagnosi e permettere un trattamento precoce.
2. **Personalizzazione del trattamento**: L'IA può analizzare i dati medici individuali di pazienti e identificare le terapie più adatte, considerando fattori come l'età, il sesso, la storia clinica e altri fattori genetici o fisici. Questo potrebbe portare a trattamenti personalizzati e più efficaci.
3. **Riduzione dei costi**: La riduzione del carico di lavoro per i medici mediante l'utilizzo dell'IA può contribuire a una maggiore efficienza nel settore sanitario, riducendo i tempi di attesa e le spese mediche. Inoltre, l'introduzione dell'AI potrebbe portare ad un minor impegno economico per i pazienti in termini di costi dei trattamenti.
4. **Precision medicine**: L'IA può aiutare a identificare le interazioni tra fattori genetici, ambientali e farmacologici che contribuiscono alla patogenesi delle malattie. Questo approccio basato sulla precisione si concentra su individui con caratteristiche specifiche per il trattamento personalizzato e più efficace.
5. **Monitoraggio continuo dei pazienti**: L'IA può essere utilizzata per monitorare i pazienti in modo continuale, rilevando eventuali cambiamenti nella salute o nelle condizioni di malattia, permettendo una gestione del caso più proactiva.
6. **Allungamento dell'esperienza vitale**: Le tecnologie basate sull'Intelligenza Artificiale potrebbero aiutare a prevenire e trattare le malattie croniche, allungando l'esperienza vita dei pazienti e riducendo il carico di malattie che colpiscono la società.
7. **Integrazione con dispositivi wearable**: L'IA può interagire con i dispositivi wearable come smartwatch o braccialelets, consentendo un monitoraggio continuo del paziente e una gestione delle condizioni mediche in tempo reale.
"""
prompt1 = BattleArenaWizLM(question, answer_a, answer_b)
res = {'question': question,
       'answer': prompt1}
database.append(res)

prompt2 = SIngleEVALWizLM(question, answer_a, "Orca13b")
res = {'question': question,
       'answer': prompt1}
database.append(res)

prompt2 = SIngleEVALWizLM(question, answer_b, "WizardLM3b")
res = {'question': question,
       'answer': prompt1}
database.append(res)

Battle between Orca13b and WizardLM13b

Fai una predizione sulle conseguenze derivanti dall'utilizzo della Intelligenza Artificiale nel campo medico.
Rispondi creando una lista in markdown evidenziando i punti chiave.

LISTA:

---

[[A]]: Assistant A provides a more comprehensive list of points related to the use of AI in medical fields,   
including potential benefits such as improved diagnosis, personalized treatment plans, reduced costs,         
precision medicine, continuous monitoring, and integration with wearable devices. This response is better than
Assistant B because it provides more detailed information on each point and covers a wider range of topics.   
Additionally, the order in which the responses were presented did not influence my decision. Therefore, I     
would choose Assistant A as the best answer.

 Generated by WizardLM-7b_unc in 0:00:06.065557

...

Fai una predizione sulle conseguenze derivanti dall'utilizzo della Intelligenza Artificiale nel campo medico.
Rispondi creando una lista in markdown evidenziando i punti chiave.

LISTA:

---

[Rating: 8]

 Generated by WizardLM-7b_unc in 0:00:01.069464

 ---

In [19]:
###############################
# save the database
###############################
import pickle
with open("WizardLM7b_JUDGE", "wb") as fp:   #Pickling
  pickle.dump(database, fp)



In [20]:

###############################
# load the database from pickle file
###############################
import pickle
with open("WizardLM7b_JUDGE", "rb") as fp:   # Unpickling
  WizardLM7b_JUDGE = pickle.load(fp)



In [21]:
WizardLM7b_JUDGE

[{'question': 'Quali sono le relazione tra AI e la filosofia per il futuro?\nRispondi seguendo questo schema:\nRIFLESSIONI: // I tuoi pensieri qui\nCRITICHE: // Le critiche alle tue considerazioni\nRISPOSTA: // La tua risposta finale',
  'answer': ' [[A]]'},
 {'question': 'Quali sono le relazione tra AI e la filosofia per il futuro?\nRispondi seguendo questo schema:\nRIFLESSIONI: // I tuoi pensieri qui\nCRITICHE: // Le critiche alle tue considerazioni\nRISPOSTA: // La tua risposta finale',
  'answer': ' [[C]]'},
 {'question': 'Quali sono le relazione tra AI e la filosofia per il futuro?\nRispondi seguendo questo schema:\nRIFLESSIONI: // I tuoi pensieri qui\nCRITICHE: // Le critiche alle tue considerazioni\nRISPOSTA: // La tua risposta finale',
  'answer': ' [[C]]'},
 {'question': 'Quali sono le relazione tra AI e la filosofia per il futuro?\nRispondi seguendo questo schema:\nRIFLESSIONI: // I tuoi pensieri qui\nCRITICHE: // Le critiche alle tue considerazioni\nRISPOSTA: // La tua rispo